# Imports

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Start a Server

In [ ]:
import subprocess

# Global list to keep track of all subprocesses
notebook_subprocesses = []

In [ ]:
# Math Server (port 58000)
proc = subprocess.Popen(["python", "math_server.py"])
notebook_subprocesses.append(proc)

# Start another server

In [ ]:
# Weather Server (port 58001)
proc = subprocess.Popen(["python", "weather_server.py"])
notebook_subprocesses.append(proc)

# The Client

In [ ]:
from client import MultiServerClient

# The host

In [ ]:
import json
from openai import AsyncOpenAI

openai_key = "your-api-key-here"  # Replace with your key

# Method to query the llm
async def query_openai(prompt):
    llm_client = AsyncOpenAI(api_key=oai_key)
    return await llm_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

async def run_host_query(user_input: str):

    endpoints = {
        "math_server": "http://127.0.0.1:58000/sse",
        "weather_server": "http://127.0.0.1:58001/sse"
    }

    # Initialize connections to all servers
    mcp_client = MultiServerClient(endpoints)
    print("Connect to servers...")
    await mcp_client.connect_all()

    # Get all tools from all servers
    tools_by_server = await mcp_client.list_all_tools()
    tool_summary = []
    for server, tools in tools_by_server.items():
        for tool in tools:
            tool_summary.append(f"server: {server}, tool: {tool.name}, description: {tool.description} input schema: {tool.inputSchema}")
    print("Tool summaries:", tool_summary)

    # Define a simple prompt which tells the llm which tools are available and how to structure the response
    prompt = f"""
You are a tool routing assistant. Choose the best tool for the user query. Available tools:

{chr(10).join(tool_summary)}

Given the user query: "{user_input}"
Respond only with a JSON object like:
{{"server": "math_server", "tool": "add", "args": {{"a": 3, "b": 5}}}}
"""

    # Query the llm
    response = await query_openai(prompt)

    # Extract the llm response content
    raw_content = response.choices[0].message.content

    # Clean up markdown if present
    cleaned_content = raw_content.strip().strip("```json").strip("```").strip()

    # Parse JSON to see which tool was chosen
    parsed = json.loads(cleaned_content)
    print("LLM chose:", parsed)

    # Call the tool via the MCP client
    result = await mcp_client.call(parsed["server"], parsed["tool"], parsed["args"])
    print("Tool result:", result)

    # Disconnect all
    await mcp_client.disconnect_all()

In [ ]:
await run_host_query("What's the weather like in Berlin?")

In [ ]:
await run_host_query("How many hours do 5 days have?")

# Stop Servers

In [ ]:
for proc in notebook_subprocesses:
    proc.terminate()  # or proc.kill() for force
notebook_subprocesses.clear()